# Structured Pruning of a Fully-Connected Keras Model

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/example_keras_pruning_mnist.ipynb) 

Welcome to this tutorial, where we will guide you through training, pruning, and retraining a fully connected Keras model. We'll begin by constructing and training a simple neural network using the Keras framework. Following this, we will introduce and apply model pruning using MCT to reduce the size of our network. Finally, we'll retrain our pruned model to recover its degraded performance due to the pruning process.


## Installing TensorFlow and Model Compression Toolkit

We start by setting up our environment by installing TensorFlow and Model Compression Toolkit and importing them.

In [ ]:
!pip install model-compression-toolkit 
!pip install tensorflow

In [27]:
import tensorflow as tf
import tensorflow_datasets as tfds
import model_compression_toolkit as mct

## Loading and Preprocessing MNIST

Let's create a function to retrive the train and test parts of MNIST dataset including preprocessing:

In [28]:
def load_and_preprocess_mnist():
    (ds_train, ds_test), ds_info = tfds.load(
        'mnist',
        split=['train', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )

    def normalize_img(image, label):
        return tf.cast(image, tf.float32) / 255., label

    ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.cache().shuffle(ds_info.splits['train'].num_examples).batch(128).prefetch(tf.data.AUTOTUNE)
    ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE).batch(128)

    return ds_train, ds_test


## Creating a Fully-Connected Model

In this tutorial section, we create a simple toy example of a fully connected model to demonstrate the pruning process using MCT. It consists of three dense layers with 128, 64, and 10 neurons.

Notably, MCT's structured pruning will target the first two dense layers for pruning, as these layers offer the opportunity to reduce output channels. This reduction can be effectively propagated by adjusting the input channels of subsequent layers.

Once our model is created, we compile it to prepare the model for training and evaluation.


In [29]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

## Training Dense Model on MNIST

Now, we can train our model using the dataset we load and evaluate it.

In [ ]:
# Load MNIST dataset
ds_train, ds_test = load_and_preprocess_mnist()

# Train and evaluate the model
model = create_model()
model.fit(ds_train, epochs=6, validation_data=ds_test)
model.evaluate(ds_test)

## Dense Model Properties

The model.summary() function in Keras provides a snapshot of the model's architecture, including layers, their types, output shapes, and the number of parameters.


In [ ]:
model.summary()

Let's break down what we see in our model summary:

- First Dense Layer: A fully connected layer with 128 output channels and 784 input channels.

- Second Dense Layer: A fully connected layer with 64 output channels and 128 input channels.

- Third Dense Layer: The final dense layer with 10 neurons (as per the number of MNIST classes) and 64 input channels.

The total parameters amount to 109,386, which roughly requiers 427.29 KB.

## MCT Structured Pruning

### Create TPC

Firstly, we'll set up the Target Platform Capabilities (TPC) to specify each layer's SIMD (Single Instruction, Multiple Data) size.

In MCT, SIMD plays a crucial role in channel grouping, affecting the pruning decision process based on channel importance for each SIMD group of channels.

We'll use the simplest structured pruning scenario for this demonstration with SIMD=1.

In [32]:
simd_size = 1

def get_tpc():
    tp = mct.target_platform
    default_config = tp.OpQuantizationConfig(
        simd_size=simd_size,
        activation_quantization_method=tp.QuantizationMethod.POWER_OF_TWO,
        weights_quantization_method=tp.QuantizationMethod.POWER_OF_TWO,
        activation_n_bits=None,
        weights_n_bits=None,
        weights_per_channel_threshold=None,
        enable_weights_quantization=None,
        enable_activation_quantization=None,
        quantization_preserving=None,
        fixed_scale=None,
        fixed_zero_point=None,
        weights_multiplier_nbits=None)

    default_configuration_options = tp.QuantizationConfigOptions([default_config])
    tp_model = tp.TargetPlatformModel(default_configuration_options)
    tpc = tp.TargetPlatformCapabilities(tp_model)
    return tpc



### Create a Representative Dataset

We are creating a representative dataset to guide our model pruning process for computing importance score for each channel:

In [33]:
# Create a representative dataset
ds_train_as_iter = iter(ds_train)

def representative_data_gen() -> list:
  yield [next(ds_train_as_iter)[0]]

### Create KPI

We're defining a Key Performance Indicator (KPI) to constrain the memory usage of our pruned model.

By setting a target that limits the model's weight memory to half of its original size (around 427KB), we aim to achieve a compression ratio of 50%:

In [34]:
# Create KPI to limit the pruned model weights memory to a certain KPI
dense_model_memory = 427*(2**10) # Original model weights requiers ~427KB
compression_ratio = 0.5

kpi = mct.KPI(weights_memory=dense_model_memory*compression_ratio)

### Prune Model

We're ready to execute the actual pruning using MCT's keras_pruning_experimental function. The model is pruned according to our defined KPI and using the representative dataset generated earlier.

Each channel's importance is measured using LFH (Label-Free-Hessian)
which approximates the Hessian of the loss function w.r.t model's weights.

In this example, we've used just one score approximation for efficiency. Although this is less time-consuming, it's worth noting that using multiple approximations would yield more precise importance scores in real-world applications. However, this precision comes with a trade-off in terms of longer processing times.

The result is a pruned model and associated pruning information, which includes details about the pruning masks and scores for each layer.

In [ ]:
num_score_approximations = 1

target_platform_cap = get_tpc()
pruned_model, pruning_info = mct.pruning.keras_pruning_experimental(
        model=model,
        target_kpi=kpi,
        representative_data_gen=representative_data_gen,
        target_platform_capabilities=target_platform_cap,
        pruning_config=mct.pruning.PruningConfig(num_score_approximations=num_score_approximations)
    )

### Pruned Model Properties

As before, we can use Keras model's API to observe the new architecture and details of the pruned model:

In [ ]:
pruned_model.summary()

## Retraining Pruned Model

After pruning models, it's common to observe a temporary drop in the model's accuracy. This decline directly results from reducing the model's complexity through pruning.

In [ ]:
pruned_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)
pruned_model.evaluate(ds_test)

However, to recover the performance, we retrain the pruned model, allowing it to adapt to its new, compressed architecture. The model can regain, and sometimes even surpass, its original accuracy through retraining.

In [ ]:
pruned_model.fit(ds_train, epochs=6, validation_data=ds_test)
pruned_model.evaluate(ds_test)

Copyright 2023 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
